<a href="https://colab.research.google.com/github/Aliaksandr-Borsuk/Recommender_Systems_project/blob/main/notebooks/05_user_segment_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Общая подготовка

**Цель:**
- исследовать возможность рекоммендаций для cold_users и Low-activity_users    

**Данные:**
- мастерим новые данные для исследования холодных и слабо-подогретых пользователей.)


## 01. Клонируем репозиторий,  подключаем Google Drive, устанавливаем зависимости

In [1]:
# Клонируем репозиторий и подключаем Google Drive
!rm -rf /content/Recommender_Systems_project
!git clone https://github.com/Aliaksandr-Borsuk/Recommender_Systems_project.git /content/Recommender_Systems_project

from google.colab import drive
drive.mount('/content/drive')

!pip install -q rs_datasets scikit-learn scipy

Cloning into '/content/Recommender_Systems_project'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 140 (delta 72), reused 51 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (140/140), 653.11 KiB | 3.75 MiB/s, done.
Resolving deltas: 100% (72/72), done.
Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.4/494.4 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 13.5 MB/s eta 0:00:00


## 02. Импорты

In [2]:
# Импорты
import sys
sys.path.append("/content/Recommender_Systems_project/src")

import numpy as np
import pandas as pd
import json

from datetime import datetime
from pathlib import Path
from pprint import pprint
from scipy.sparse import csr_matrix, load_npz
import pickle
import random

from sklearn.decomposition import TruncatedSVD
from rs_datasets import MovieLens

from recommender.data_io import train_test_reader
from recommender.preprocessing import prepare_ui_matrix
from recommender.metrics import model_evaluation
from recommender.results_logger import save_experiment_results

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

DATA = Path("/content/drive/MyDrive/Colab Notebooks/data/")
RAW_DATA = DATA / "raw"
PROCESSED = DATA / "processed"
RESULTS_DIR = DATA / "results"
TOP_K = 10

## 03. Загрузка полного датасета MovieLens 1M

In [3]:
# Загрузка полного датасета MovieLens 1M
ml = MovieLens('1m', path=RAW_DATA)
ratings = ml.ratings
items = ml.items

# Объединяем
df = ratings.merge(items[['item_id', 'title', 'genres']], on='item_id', how='left')
df['ts'] = pd.to_datetime(df['timestamp'], unit='s')
print(f"Полный df shape: {df.shape}")

# Информация
n_users_full = df['user_id'].nunique()
n_items_full = df['item_id'].nunique()
print(f"Пользователей: {n_users_full}, Айтемов: {n_items_full}")

Полный df shape: (1000209, 7)
Пользователей: 6040, Айтемов: 3706


# Общие замечания.
В рекомендательных системах можно выделить три категории пользователей:
1. **Cold users** (холодные пользователи).
  - - **Характеристика:** отсутствуют в train полностью, но есть в test (оставляем только с ≥10 взаимодействий для корректной оценки метрик)
    
2. **Low-activity users** (пользователи с низкой активностью).  
- - **Характеристика:** в train: мало взаимодействий (например, < 15), в test: достаточно для оценки (>= 10)   

3. **Warm users** (тёплые пользователи)
- - **Характеристика:** в train: >= 15, в test: >= 10

## 04. Делаем разбиение по времени(Time-based split)   
- точно так же как и в пкрвом ноутбуке 001_data_and_eda_1m.ipynb
- выбираем порог времени 80% . Все события до порога в train, события после в test. Это гарантирует, что ни одно событие из "будущего" не попадёт в train.

In [4]:
# колонки для сохранения
columns_to_save = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres']
# limit
time_treshold = df['ts'].quantile(q=0.8, interpolation='nearest')
print(f"Порог разбиения по времени {time_treshold}")

train_df = df[df['ts'] <= time_treshold][columns_to_save]
test_df = df[df['ts'] > time_treshold][columns_to_save]
print(f"Размеры train {train_df.shape[0]} test {test_df.shape[0]}")

users_intersection = set(test_df['user_id']) & set(train_df['user_id'])
print(f"train содержит {train_df['user_id'].nunique()} пользователей\t и {train_df.shape[0]} строк,\
\ntest содержит  { test_df['user_id'].nunique()} пользователей \t и {test_df.shape[0]} строк,\
\n{len(users_intersection)} юзера встречаются одновременно и в train и в test")

# залистим тестовых пользователей
test_users = sorted(users_intersection)

print('\ntrain ')
display(train_df.head(2))
print('\ntest ')
display(test_df.head(2))

Порог разбиения по времени 2000-12-02 14:52:18
Размеры train 800168 test 200041
train содержит 5400 пользователей	 и 800168 строк,
test содержит  1783 пользователей 	 и 200041 строк,
1143 юзера встречаются одновременно и в train и в test

train 


,user_id,item_id,rating,timestamp,title,genres
94507,635,1251,4,975768620,8 1/2 (1963),Drama
94513,635,3948,4,975768294,Meet the Parents (2000),Comedy



test 


,user_id,item_id,rating,timestamp,title,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical


### Загрузка warm_пользователей

In [5]:
#  Загрузка warm-only split из 01 ноутбука (для сравнения)
train_test_path = '/content/drive/MyDrive/Colab Notebooks/data/processed/251021_173655'
train_warm, test_warm, meta_warm = train_test_reader(train_test_path)

pprint(meta_warm, width=80, compact=False)
print(f"\nТолько Warm test users: {meta_warm['n_test_users']}")

{'columns': ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres'],
 'created_at': '2025-10-21T17:37:00.607645',
 'min_test_interactions': 10,
 'min_train_interactions': 5,
 'n_items': 3662,
 'n_test_users': 836,
 'n_train_users': 5392,
 'test_shape': [94842, 6],
 'time_treshold': '2000-12-02T14:52:18',
 'train_shape': [800142, 6]}

Только Warm test users: 836


### Cold users (холодные пользователи).
( для обучения моделей можно использовать тот же train, что и для тепленьких пользователей.)


In [6]:
# оставляем в test только item'ы, которые есть в train
# ни одна модель не сможет порекомендовать что-то невиданое
train_items = set(train_df['item_id'].unique())
good_test_df = test_df[test_df['item_id'].isin(train_items)]


# требования к юзерам в тест по min кол-ву оценок
k=10
train_n_rait = train_df['user_id'].value_counts()
test_n_rait = good_test_df['user_id'].value_counts()

train_users = set(train_n_rait.index)
valid_test_users = set(test_n_rait[test_n_rait >= k].index)

cold_users = valid_test_users.difference(train_users)

cold_test_df = good_test_df[good_test_df['user_id'].isin(cold_users)]


print(f"Осталось холодных пользователей в test: {len(cold_users)}")
print(f"cold_test shape: {cold_test_df.shape}")

Осталось холодных пользователей в test: 640
cold_test shape: (95478, 6)


In [ ]:
# сохраним Cold_test
# папка для сохранения данных
time_stamp = datetime.now().strftime('%y%m%d_%H%M%S')
out_dir = PROCESSED/f'{time_stamp}'
out_dir.mkdir(parents=True, exist_ok=True)

# сохраняем сами данные
cold_test_df.to_csv(out_dir / 'cold_test_df.csv', index=False)


# мета-информация
meta = {
    'created_at': datetime.now().isoformat(),
    'test_shape': cold_test_df.shape,
    'time_treshold': time_treshold.isoformat(),
    'n_test_users': int(cold_test_df['user_id'].nunique()),
    'n_items': int(cold_test_df['item_id'].nunique()),
    'min_cold_test_df_interactions': 10,
    'columns': list(cold_test_df.columns)
}

with open(out_dir / 'meta.json', 'w', encoding='utf8') as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)

print(f'cold_test_df сохранён в {out_dir.absolute()}')

cold_test_df сохранён в /content/drive/MyDrive/Colab Notebooks/data/processed/251221_175103


### Low-activity users (пользователи с низкой активностью).
- найдём честно юзеров с низкой активностью


In [7]:
# max и min кол-во оценок у каждого юзера в train и в test
n, k = 15, 10
train_n_rait = train_df['user_id'].value_counts()
test_n_rait = good_test_df['user_id'].value_counts()

low_activity_users = (set(train_n_rait[train_n_rait < n].index)
              & set(test_n_rait[test_n_rait >= k].index))

low_activity_test_df = good_test_df[good_test_df['user_id'].isin(low_activity_users)]


print(f"Осталось low_activity пользователей в test: {len(low_activity_users)}")
print(f"test shape: {low_activity_test_df.shape}")

Осталось low_activity пользователей в test: 77
test shape: (8609, 6)


In [ ]:
# сохраним честный low_activity_test
# папка для сохранения данных
time_stamp = datetime.now().strftime('%y%m%d_%H%M%S')
out_dir = PROCESSED/f'{time_stamp}'
out_dir.mkdir(parents=True, exist_ok=True)

# сохраняем сами данные
low_activity_test_df.to_csv(out_dir / 'honest_low_activity_test_df.csv', index=False)


# мета-информация
meta = {
    'created_at': datetime.now().isoformat(),
    'test_shape': low_activity_test_df.shape,
    'time_treshold': time_treshold.isoformat(),
    'n_test_users': int(low_activity_test_df['user_id'].nunique()),
    'n_items': int(low_activity_test_df['item_id'].nunique()),
    'min_cold_test_df_interactions': 10,
    'columns': list(low_activity_test_df.columns)
}

with open(out_dir / 'meta.json', 'w', encoding='utf8') as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)

print(f'честный low_activity_test_df сохранён в {out_dir.absolute()}')

честный low_activity_test_df сохранён в /content/drive/MyDrive/Colab Notebooks/data/processed/251221_181101


Образовалась небольшая проблема - 77 пользователей дадут нестабильную оценку моделей, а нам бы этого не хотелось...
Выход - смастерим дополнительно искусствено ~10%(от тёплых пользователей)  low_activity пользователей только для оценки моделей

In [8]:
# увеличим n
n=50
new_low_activity_users = (set(train_n_rait[train_n_rait < n].index)
              & set(test_n_rait[test_n_rait >= k].index))
# сформируем test для оценки предсказаний на low_activity пользователях
new_low_activity_test = good_test_df[good_test_df['user_id'].isin(new_low_activity_users)]


print(f"Имеем new_low_activity пользователей : {len(new_low_activity_users)}")
print(f"new_test shape: {new_low_activity_test.shape}")

low_train_rows = []
new_low_activity_train = train_df[train_df['user_id'].isin(new_low_activity_users)]
new_train = train_df[~train_df['user_id'].isin(new_low_activity_users)]

train_grouped = new_low_activity_train.groupby('user_id')

random.seed(RANDOM_STATE)
for user in new_low_activity_users:
   train_user = train_grouped.get_group(user)
   # обрезаем историю пользователя до рандомной маленькой длины
   train_max_interactions = random.randint(4,14)
   train_user_sorted = train_user.sort_values('timestamp')
   train_user_tail = train_user_sorted.tail(train_max_interactions)
   new_train = pd.concat([new_train, train_user_tail], ignore_index=True)

# проверка на дубликаты
assert new_train.duplicated().sum() == 0, "Ahtung!!! Есть дубликаты в new_train!"
# все low-activity пользователи попали в new_train
users_in_new_train = set(new_train['user_id'])
assert new_low_activity_users.issubset(users_in_new_train), "Ahtung!!! Не все low-activity пользователи в new_train"
new_train

Имеем new_low_activity пользователей : 191
new_test shape: (19761, 6)


,user_id,item_id,rating,timestamp,title,genres
0,636,2054,4,975752834,"Honey, I Shrunk the Kids (1989)",Adventure|Children's|Comedy|Fantasy|Sci-Fi
1,636,589,5,975752617,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
2,636,1261,5,975752226,Evil Dead II (Dead By Dawn) (1987),Action|Adventure|Comedy|Horror
3,636,3016,3,975752226,Creepshow (1982),Horror
4,636,1274,2,975752617,Akira (1988),Adventure|Animation|Sci-Fi|Thriller
...,...,...,...,...,...,...
797450,5630,2889,3,959020754,"Mystery, Alaska (1999)",Comedy
797451,5630,2124,3,959020830,"Addams Family, The (1991)",Comedy
797452,5630,2771,4,959020850,Brokedown Palace (1999),Drama
797453,5630,3157,3,959020850,Stuart Little (1999),Children's|Comedy


In [ ]:
# папка для сохранения данных
time_stamp = datetime.now().strftime('%y%m%d_%H%M%S')
out_dir = PROCESSED/f'{time_stamp}'
out_dir.mkdir(parents=True, exist_ok=True)

# сохраняем сами данные
new_train.to_csv(out_dir / 'new_low_activity_train.csv', index=False)
new_low_activity_test.to_csv(out_dir / 'new_low_activity_test.csv', index=False)

# мета-информация
meta = {
    'created_at': datetime.now().isoformat(),
    'train_shape': new_train.shape,
    'test_shape': new_low_activity_test.shape,
    'time_treshold': time_treshold.isoformat(),
    'n_train_users': int(new_train['user_id'].nunique()),
    'n_test_users': int(new_low_activity_test['user_id'].nunique()),
    'n_items': int(new_train['item_id'].nunique()),
    'max_train_interactions': 15,
    'min_test_interactions': 10,
    'columns': list(new_train.columns)
}

with open(out_dir / 'meta.json', 'w', encoding='utf8') as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)

print(f'low_activity train/test сохранён в {out_dir.absolute()}')

low_activity train/test сохранён в /content/drive/MyDrive/Colab Notebooks/data/processed/251221_180520


### **Выводы:**
**Cold users (холодные пользователи).**  
- **Количество**
- - 640 пользователей
- **Особенности:**
- - Коллаборативные модели (SVD, ALS, EASE, KNN) не могут сделать персонализированные рекомендации
- - Единственный вариант - Popularity или контент-based подходы  

**Low-activity users (пользователи с низкой активностью)**  
- **Количество**
- - 77 пользователей
- - увеличили до 191 но оценка будет немного искажена
- **Особенности:**
- Модели работают, но качество снижено
- Показывает устойчивость алгоритмов к sparse-профилям  

**Warm users (тёплые пользователи)**
- **Количество**
- - 836 пользователей (основной test из 01)
- **Особенности:**
- - все  модели работают эффективно
- -  основной набор для сравнения моделей
- - гарантирует стабильные метрики

## 05. Оценка на Cold-start (640 пользователей)
- SVD(и другие модели основаные на прошлых взаимодействиях) не может рекомендовать cold-пользователям - только Popularity

In [9]:
# только Popularity
k = TOP_K
popular_items = train_warm['item_id'].value_counts().head(k).index.to_list()
test_dict   = cold_test_df.groupby('user_id')['item_id'].apply(set).to_dict()
rec_items = {user : popular_items for user in test_dict}
all_items = set(train_warm['item_id'].unique())

result = model_evaluation(rec_items, test_dict, all_items, k=k, model_name='Most_Popular')
display(result)

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
Most_Popular,0.935937,0.461875,0.05017,0.486267,0.358343,0.002731


## Оценка на Low-activity

### Most_Popularity

In [10]:
# Popularity для честных low_activity
k = TOP_K
popular_items = train_warm['item_id'].value_counts().head(k).index.to_list()
test_dict   = low_activity_test_df.groupby('user_id')['item_id'].apply(set).to_dict()
rec_items = {user : popular_items for user in test_dict}
all_items = set(train_warm['item_id'].unique())

result = model_evaluation(rec_items, test_dict, all_items, k=k, model_name='Most_Popular_honest_low_act')
display(result)

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
Most_Popular_honest_low_act,0.831169,0.290909,0.035137,0.31713,0.203309,0.002731


In [11]:
# Popularity для созданных low_activity
popular_items = new_train['item_id'].value_counts().head(k).index.to_list()
test_dict   = new_low_activity_test.groupby('user_id')['item_id'].apply(set).to_dict()
rec_items = {user : popular_items for user in test_dict}
all_items = set(new_train['item_id'].unique())

result = model_evaluation(rec_items, test_dict, all_items, k=k, model_name='Most_Popular_honest_low_act')
display(result)

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
Most_Popular_honest_low_act,0.743455,0.249215,0.030692,0.259217,0.160005,0.002731


### truncated_SVD

In [12]:
def recommend_with_svd(svd_model, train_matrix, val_users, topn=10) -> dict[int, list[int]]:
    """
    Генерация рекомендаций для всех пользователей на основе SVD.
    """
   # Эмбеддинги пользователей - только для нужных пользователей
    X_svd = svd_model.transform(train_matrix[val_users])  # (n_val_users, n_components)

    components_ = svd_model.components_  # (n_components, n_items)

    # Матричное умножение для всех пользователей сразу
    user_scores = X_svd @ components_  # (n_val_users, n_items)

    # Маска уже просмотренных айтемов
    seen_mask = train_matrix[val_users].toarray().astype(bool)  # (n_val_users, n_items)
    user_scores[seen_mask] = -np.inf

    # Получаем топ-N индексов для всех пользователей сразу
    top_indices = np.argsort(-user_scores, axis=1)[:, :topn]  # (n_val_users, topn)

    # Создаем словарь рекомендаций
    recs = {user_id: indices.tolist() for user_id, indices in zip(val_users, top_indices)}

    return recs

### для честных low_activity

In [13]:
## Получение метрик  для честных low_activity
explicit_train_matrix, user2index, item2index =\
    prepare_ui_matrix(
                  train_df,
                  user_col='user_id',
                  item_col='item_id',
                  rating_col='rating',
                  implicit=False,       # работаем с explicit рейтингами
                  threshold=None,       # не фильтруем по порогу что бы матрица
                  center= None,
                  normalize=None
              )

In [16]:
# создаём модель
svd = TruncatedSVD(n_components=5, random_state=RANDOM_STATE, n_iter=42 )
svd.fit(explicit_train_matrix)

TruncatedSVD(n_components=5, n_iter=42, random_state=42)

In [17]:
# заменяем реальные ID на индексы
test_mapped = low_activity_test_df.assign(
    user_id = low_activity_test_df["user_id"].map(user2index),
    item_id = low_activity_test_df["item_id"].map(item2index)
)
assert test_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
test_items = test_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(train_warm['item_id'].map(item2index).dropna().astype(int).unique())

In [18]:
# рекомендации
svd_recs = recommend_with_svd(svd, explicit_train_matrix, list(test_items.keys()), topn=TOP_K)
svd_results = model_evaluation(svd_recs,test_items, all_items, 10, 'trancated_svd_n_comp=5_n_iter=42')
svd_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
trancated_svd_n_comp=5_n_iter=42,0.792208,0.311688,0.041488,0.323326,0.221143,0.025396


### для расширеных low_activity

In [ ]:
## Получение метрик  для созданных low_activity
explicit_train_matrix, user2index, item2index =\
    prepare_ui_matrix(
                  new_train,
                  user_col='user_id',
                  item_col='item_id',
                  rating_col='rating',
                  implicit=False,       # работаем с explicit рейтингами
                  threshold=None,       # не фильтруем по порогу что бы матрица
                  center= None,
                  normalize=None
              )

In [ ]:
# создаём модель
svd = TruncatedSVD(n_components=5, random_state=RANDOM_STATE, n_iter=42 )
svd.fit(explicit_train_matrix)

TruncatedSVD(n_components=5, n_iter=42, random_state=42)

In [ ]:
# заменяем реальные ID на индексы
test_mapped = new_low_activity_test.assign(
    user_id = new_low_activity_test["user_id"].map(user2index),
    item_id = new_low_activity_test["item_id"].map(item2index)
)
assert test_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
test_items = test_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(new_train['item_id'].map(item2index).dropna().astype(int).unique())

In [ ]:
# рекомендации
svd_recs = recommend_with_svd(svd, explicit_train_matrix, list(test_items.keys()), topn=TOP_K)
svd_results = model_evaluation(svd_recs,test_items, all_items, 10, 'trancated_svd_n_comp=5_n_iter=42')
svd_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
trancated_svd_n_comp=5_n_iter=42,0.764398,0.271204,0.037543,0.273483,0.171011,0.036319


# **Итого:**

## **Наблюдения**:
- **Warm-only:** SVD значительно превосходит Popularity по NDCG и MAP.
- **Cold-start**: только Popularity применима; SVD не работает.
- **Low-activity (77):** качество SVD падает, но остаётся выше Popularity.(только hit_rate@10 у popylarity выше)
- **Расширенный low-activity (191):** SVD лучше Popularrity по всем метрикам, что подтверждает устойчивость SVD к sparse-профилям."
## **Выводы**:
- Использовать SVD как основную модель для warm/low-activity пользователей
- Для cold-start использовать Popularity или контент-based.